In [1]:
import os
import astroNN
from astroNN import datasets
from astroNN.apogee import allstar, visit_spectra, combined_spectra, gap_delete, apogee_continuum, bitmask_boolean
from astroNN.models import load_folder
from astroNN.apogee.chips import wavelength_solution, chips_split

from astropy.io import fits
from astropy.wcs import WCS
from tensorflow.keras import utils
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pickle
import traceback
import h5py as h5

%matplotlib inline
import matplotlib.ticker as ticker

2023-06-15 14:45:16.890254: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-15 14:45:18.969419: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
geir_path = "/geir_data/scr/petershi" #should save work here 
sdssv_path = "/yngve_data/sdssv/ipl-2/spectro/astra/0.3.0/v6_0_9-1.0/spectra/star" #path for SDSSV spectra
sdssiv_path = "/yngve_data/sdss/apogee/dr17/apogee/spectro/redux/dr17/stars/apo25m"

xmatch_path = "/yngve_data/sdssv/ipl-2/spectro/astra/0.3.0/v6_0_9-1.0/aux_xmatch/ipl-1_dr17_lite_230517_byYS.fits"
catlist_path = "/yngve_data/sdssv/ipl-2/spectro/astra/0.3.0/v6_0_9-1.0/aux_xmatch/allStar-TIC_v8-APOGEE-0.2.6-v6_0_9-1.0.fits"

In [3]:
def getPathList(path_0):

    l = []

    for root, dirs, files in tqdm(os.walk(path_0, topdown=False)):
        for name in files:
            l.append(os.path.join(root, name))
        # for name in dirs:
        #     l.append(os.path.join(root, name))
    return (l, path_0)

In [4]:
def getPath(s, l):
    for elem in l[0]:
        if str(s) in elem:
            return elem

    print("File " + str(s) + " not found in " + str(l[1]))
    return None

In [5]:
def getSDSSVData(hdul):

    snr = hdul[3].data["SNR"]
    ra = hdul[0].header["RA"]
    dec = hdul[0].header["DEC"]
    flux = hdul[3].data["FLUX"]
    flux_err = hdul[3].data["E_FLUX"]
    bitmask = hdul[3].data["BITMASK"]

    rd = {}

    rd["SNR"] = snr
    rd["RA"] = ra
    rd["DEC"] = dec
    rd["FLUX"] = flux
    rd["FLUX_ERR"] = flux_err
    rd["BITMASK"] = bitmask

    return rd 

In [6]:
sdssv_files = getPathList(sdssv_path)

10101it [00:22, 447.09it/s]


In [7]:
example = sdssv_files[0][0]
ex = fits.open(example)

In [8]:
ex.info()

Filename: /yngve_data/sdssv/ipl-2/spectro/astra/0.3.0/v6_0_9-1.0/spectra/star/00/00/mwmStar-0.3.0-27021598754250000.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      59   ()      
  1  BOSS/APO      1 BinTableHDU     19   0R x 0C   []   
  2  BOSS/LCO      1 BinTableHDU     19   0R x 0C   []   
  3  APOGEE/APO    1 BinTableHDU     83   1R x 11C   [1E, 8575E, 8575E, 8575E, 8575J, 8575E, 8575E, 21E, 50E, 1E, 1L]   
  4  APOGEE/LCO    1 BinTableHDU     19   0R x 0C   []   


In [9]:
print(repr(ex[3].header))

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =               206093 / length of dimension 1                          
NAXIS2  =                    1 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                   11 / number of table fields                         
                                                                                
        METADATA                                                                
EXTNAME = 'APOGEE/APO'         / Short extension name                           
OBSRVTRY= 'APO     '           / Observatory name                               
INSTRMNT= 'APOGEE  '        

In [10]:
hdul = fits.open(catlist_path)

In [11]:
hdul.info()

Filename: /yngve_data/sdssv/ipl-2/spectro/astra/0.3.0/v6_0_9-1.0/aux_xmatch/allStar-TIC_v8-APOGEE-0.2.6-v6_0_9-1.0.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU    287   153700R x 131C   [K, D, D, K, K, K, 17A, K, K, 19A, K, 8A, K, 12A, 10A, 17A, K, 19A, 8A, K, 4A, 10A, 10A, D, D, D, D, 6A, D, D, 5A, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, 19A, D, D, D, D, D, D, D, D, D, D, D, 5A, D, 5A, D, D, D, D, D, D, D, D, D, D, D, D, 5A, D, D, D, D, D, D, K, D, 5A, D, D, D, 9A, D, D, D, D, D, D, D, D, D, D, D, D, 6A, D, D, 6A, D, D, K, D, D, D, 8A, D, 8A, D, 15A, D, D, D, K, K, K, K, K]   


In [12]:
cat = hdul[1].data["catalogid"]

In [13]:
h5_path = os.path.join(geir_path, "serialized", "SDSS-V Stars.hdf5")
hf = h5.File(h5_path, 'a')

for c in tqdm(cat):
    
    try:
        cpath = getPath(c, sdssv_files)

        if cpath is not None:
            
            # for SDSS-V data 
            cfile = fits.open(cpath, memmap = False)

            try:
                cdata = getSDSSVData(cfile)
            except Exception as e: 
                print(traceback.format_exc())
                cdata = None

            cfile.close()
            for hdu in cfile: #close files 
                del hdu.data
            del cfile

            if cdata is not None:
                
                grp = hf.create_group(str(c))

                for key in list(cdata.keys()):
                    dset = grp.create_dataset(key, data = cdata[key])

                

    except Exception as e:
        print(e)

        print(c)

hf.close()

  0%|          | 722/153700 [03:37<11:44:51,  3.62it/s]

File 4215415336 not found in /yngve_data/sdssv/ipl-2/spectro/astra/0.3.0/v6_0_9-1.0/spectra/star


  1%|          | 1385/153700 [07:03<11:41:23,  3.62it/s]

File 4223279602 not found in /yngve_data/sdssv/ipl-2/spectro/astra/0.3.0/v6_0_9-1.0/spectra/star


  1%|▏         | 2097/153700 [10:37<14:53:04,  2.83it/s]

...

100%|█████████▉| 153490/153700 [13:25:34<01:38,  2.14it/s]

File 27021599489454080 not found in /yngve_data/sdssv/ipl-2/spectro/astra/0.3.0/v6_0_9-1.0/spectra/star


100%|█████████▉| 153493/153700 [13:25:35<01:37,  2.12it/s]

File 27021599489454091 not found in /yngve_data/sdssv/ipl-2/spectro/astra/0.3.0/v6_0_9-1.0/spectra/star


100%|██████████| 153700/153700 [13:27:03<00:00,  3.17it/s]


In [15]:
hf = h5.File(h5_path, 'r')

len(list(hf.keys()))

152024

In [16]:
hf.close()